In [1]:
# --- Load GPyOpt
from GPyOpt.methods import BayesianOptimization
import Utils
import Optimizer as opt
import NeuralNetworks as Net
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random

In [2]:
def Hyp_Optim_SCA(array):
    """ Function to Obtain the Hyper Parameter Optimization """
    
    C = 0.001
    SSE = 0.0
    rho_eps = 0.0
    SS,rho, tau  = array[0]
    #initialize
    device,sequence_length,input_size,hidden_size,num_layers,num_classes,batch_size = Utils.init_hyper_param()
    train = torchvision.datasets.MNIST(root="C://Users//conne//Master Thesis",train=True, transform=transforms.ToTensor())
    
    print ('Training Started')
    #train the NN
    iters=600
    #loss_history = torch.zeros(iters)
    #Initialize the Neural Network to the Same Point with normal distribution
    lstm_model,parameters = Net.random_initialization_neural_networks(input_size,hidden_size,num_layers,num_classes,device)
    #Apply SCA Ridge and Optimize the model
    final_lstm,err_lstm,grad_lstm,message = opt.sca_ridge(lstm_model,parameters,train,batch_size,sequence_length,input_size,max_it=iters,
                                                      step_size=2**SS,C=C,tau=2**tau,rho_eps=rho_eps,step_size_eps=SSE,rho=2**rho)
    
    print ('Calculating Valid Accuracy')
    
    if 'Singular matrix' in message:
        print ('Accuracy is zero, Singular Matrix')
        return 0.0
    else:
        #cal accuracy on test data set
        acc = Utils.Valid_Accuracy(final_lstm,train,batch_size,sequence_length,input_size)
        print ('Accuracy is :',acc, '%')
        return acc
    
    
def Hyp_Optim_Adam(array):
    """ Function to Obtain the Hyper Parameter Optimization """
    
    C = 0.001
    SS,b_1,b_2,eps_  = array[0]
    #initialize
    device,sequence_length,input_size,hidden_size,num_layers,num_classes,batch_size = Utils.init_hyper_param()
    train = torchvision.datasets.MNIST(root="C://Users//conne//Master Thesis",train=True, transform=transforms.ToTensor())
    
    print ('Training Started')
    #train the NN
    iters=600
    #loss_history = torch.zeros(iters)
    #Initialize the Neural Network to the Same Point with normal distribution
    lstm_model,parameters = Net.random_initialization_neural_networks(input_size,hidden_size,num_layers,num_classes,device)
    #Apply SCA Ridge and Optimize the model
    final_lstm,err_lstm,grad_lstm = opt.adam(lstm_model,train,parameters,batch_size,sequence_length,input_size,max_it=iters,
                                                      step_size=SS,b1=b_1,b2=b_2,eps=eps_,C=C)
    
    print ('Calculating Valid Accuracy')
    #cal accuracy on test data set
    acc = Utils.Valid_Accuracy(final_lstm,train,batch_size,sequence_length,input_size)
    print ('Accuracy is :',acc, '%')
    return acc

In [ ]:
'''
Set Up for Hyper Parameter Optimization of SCA
'''
domain = [{'name': 'SS', 'type': 'continuous', 'domain': (-10,10)},
          {'name': 'rho', 'type': 'continuous', 'domain': (-10,10)},
          {'name': 'tau', 'type': 'continuous', 'domain': (-10,10)}
          ]
inp = np.array([-3.65240772, -0.72593248, -5.28125507,
               -0.5145731728297583 , -0.15200309344504997, -2.321928094887362]).reshape(2,3)
out = np.array([82.17,76.37]).reshape(2,1)
myBopt = BayesianOptimization(f=Hyp_Optim_SCA, domain=domain , model_type='GP' , X=inp, Y=out, maximize=True )
myBopt.run_optimization(max_iter=20)

Training Started
Calculating Valid Accuracy
Accuracy is : 51.93 %
Training Started
Calculating Valid Accuracy
Accuracy is : 58.41 %
Training Started


In [ ]:
#5,06

In [93]:
Hyp_Optim_SCA(inp)

Training Started
Calculating Valid Accuracy
Accuracy is : 82.17 %


82.17

In [ ]:
#Max Accuracy on Test Score, SCA: 82 %, Adam, 84 %

In [39]:
'''
Set Up for Hyper Parameter Optimization of Adam
'''
domain = [{'name': 'SS', 'type': 'continuous', 'domain': (-10,10)},
          {'name': 'b1', 'type': 'continuous', 'domain': (0,1)},
          {'name': 'b2', 'type': 'continuous', 'domain': (0,1)},
          {'name': 'eps', 'type': 'continuous', 'domain': (-10,10)}
          ]
inp_adam = np.array([0.01,0.9,0.999,10**-8,4.05234227, 
                     0.70149578, 0.65954886, 0.42508586,
                     0.05,0.95,0.9,10**-7,
                    3.34193507, 0.43180648, 0.30817403, 0.39198932,
                    5.21584742, 0.64755972, 0.0, 0.60235577]).reshape(5,4)
out_adam = np.array([82.9,70.73,69.0,67.62,70.25]).reshape(5,1)
myBopt = BayesianOptimization(f=Hyp_Optim_Adam, domain=domain , model_type='GP' , X=inp_adam, Y=out_adam, maximize=True )
myBopt.run_optimization(max_iter=50)

Training Started
Calculating Valid Accuracy
Accuracy is : 58.04 %
Training Started
Calculating Valid Accuracy
Accuracy is : 54.87 %
Training Started
Calculating Valid Accuracy
Accuracy is : 56.45 %
Training Started
Calculating Valid Accuracy
Accuracy is : 67.62 %
Training Started
Calculating Valid Accuracy
Accuracy is : 51.56 %
Training Started
Calculating Valid Accuracy
Accuracy is : 58.89 %
Training Started
Calculating Valid Accuracy
Accuracy is : 53.27 %
Training Started
Calculating Valid Accuracy
Accuracy is : 42.84 %
Training Started
Calculating Valid Accuracy
Accuracy is : 54.98 %
Training Started
Calculating Valid Accuracy
Accuracy is : 45.87 %
Training Started
Calculating Valid Accuracy
Accuracy is : 40.25 %
Training Started
Calculating Valid Accuracy
Accuracy is : 33.77 %
Training Started
Calculating Valid Accuracy
Accuracy is : 38.9 %
Training Started
Calculating Valid Accuracy
Accuracy is : 32.94 %
Training Started
Calculating Valid Accuracy
Accuracy is : 36.76 %
Training St

In [77]:
inp_adam = np.array([0.012,0.9,0.999,10**-8]).reshape(1,4)

In [79]:
Hyp_Optim_Adam(inp_adam)

Training Started
Calculating Valid Accuracy
Accuracy is : 84.18 %


84.18